## Marketing Demand Gen - Lead Scoring Pipeline

This notebook will show you how to AI can:
1. Classify the quality and qualification of demand gen leads and 
2. Score the leads for urgency

VIEW: Example Demand Gen form: https://drive.google.com/file/d/1QNl0QfGbE0ucLM-nFPfsZ_h4dy64i9oY/view

In [ ]:
-- Edit as needed
CREATE DATABASE IF NOT EXISTS AI_DEMO;
CREATE SCHEMA IF NOT EXISTS MARKETING;

Create the table `MARKETING_LEADS` from the file: MarketingMockupData.csv

In [ ]:
-- Preview our data to ensure it's been loaded
select * from AI_DEMO.MARKETING.MARKETING_LEADS LIMIT 10;

### CLASSIFY_TEXT
Classifies free-form text into categories that you provide.

SYNTAX: SNOWFLAKE.CORTEX.CLASSIFY_TEXT( <input> , <list_of_categories>, [ <options> ] )

https://docs.snowflake.com/en/sql-reference/functions/classify_text-snowflake-cortex

In [ ]:
select d.*
, snowflake.cortex.CLASSIFY_TEXT(

    -- INPUT TEXT (concat several fields into a string)
    concat_ws(' '
        ,'First Name:', d.first_name
        ,'Last Name:', d.last_name
        ,'Job Title:', d.title
        ,'Company:', d.company )
        
    --CATEGORIES (Array that represents 2-100 categories)
    , [
        {
        'label': 'VALID',
        'description': 'A form fill that appears to be legitimate data from a valid persona.'
        },
        {
        'label': 'SPAM',
        'description': 'A form fill that appears to contain test accounts, junk, spam, scams, jokes, or nonsensical entries that cannot be worked by sales. This category should also include students and people without a valid company.'
        }
      ]
      
    --OPTIONS (A short explanation of the text classification task)
    , {'task_description': 'We are a B2B SAAS company. Return a classification for this text entered to a demand gen form on our website to register for a marketing event.'}
    
 ) as classification_output_raw
 , trim(classification_output_raw:label, '"') as classification_output
from AI_DEMO.MARKETING.MARKETING_LEADS d
;


In [ ]:
select d.*
, snowflake.cortex.CLASSIFY_TEXT(

    -- INPUT TEXT (concat several fields into a string)
    concat_ws(' '
        ,'First Name:', d.first_name
        ,'Last Name:', d.last_name
        ,'Job Title:', d.title
        ,'Company:', d.company )

    --CATEGORIES (Array that represents 2-100 categories)
    , [
        {
        'label': 'HARMFUL',
        'description': 'A form fill contains language not appropriate in a business context.'
        },{
        'label': 'NOT HARMFUL'
        }
      ]

    --OPTIONS (A short explanation of the text classification task)
    , {'task_description': 'We are a B2B SAAS company. Return a classification for this text entered to a demand gen form on our website to register for a marketing event.'}
    
  ) as classification_output_raw
  , trim(classification_output_raw:label, '"')
from AI_DEMO.MARKETING.MARKETING_LEADS d;


In [ ]:
select d.*
, snowflake.cortex.CLASSIFY_TEXT(

    -- INPUT TEXT (concat several fields into a string)
    concat_ws(' ','Job Title:', d.title )
    
    --CATEGORIES (Array that represents 2-100 categories)    
    , [
        {
        'label': 'CXO',
        'description': 'Any title that is a company C-level executive or founder.'
        },{
        'label': 'VP+',
        'description': 'Any title that is a company executive below C-level like a VP, President, or managing director.'
        },{
        'label': 'Director',
        'description': 'Any title that is director level or head of a department.'
        },{
        'label': 'Manager',
        'description': 'Any title that relates to managers or team leads.'
        },{
        'label': 'IC',
        'description': 'Any title that relates to an individual contributor.'
        },{
        'label': 'JUNK',
        'description': 'Any title seems like junk, spam, scams, jokes, or nonsensical entries that cannot be worked by sales'
        }
      ]

    --OPTIONS (A short explanation of the text classification task)
    ,
      {'task_description': 'We are a B2B SAAS company. Use this data to classify job titles into seniority groupings. Consider all parts of the title and be careful of how the meaning changes based on parathesis or other punctuation.'}
      
      ) as classification_output_raw
      , trim(classification_output_raw:label, '"') as classification_output
from AI_DEMO.MARKETING.MARKETING_LEADS d;


In [ ]:
select d.*
, snowflake.cortex.CLASSIFY_TEXT(

    -- INPUT TEXT (concat several fields into a string)
        concat_ws(' ','Company:', d.company )
    
    --CATEGORIES (Array that represents 2-100 categories)    
    , [
        {
        'label': 'VALID',
        'description': 'The company name appears to be valid.'

        },{
        'label': 'NOT VALID',
        'description': 'A company name seems like junk, spam, scams, jokes, or nonsensical entries that cannot be recognized by sales'
        }
      ]

    --OPTIONS (A short explanation of the text classification task)
    ,
      {'task_description': 'We are a B2B SAAS company. Use this data to classify provided company names as being likely valid or not.'}
      
      ) as classification_output_raw
      , trim(classification_output_raw:label, '"') as classification_output
from AI_DEMO.MARKETING.MARKETING_LEADS d;


In [ ]:
select d.*
, snowflake.cortex.CLASSIFY_TEXT(

    -- INPUT TEXT (concat several fields into a string)
    concat_ws(' '
        ,'Job Title:', d.title
        ,'Company:', d.company )
        
    --CATEGORIES (Array that represents 2-100 categories)    
    ,[
        {
        'label': 'High',
        'description': 'The lead has a decision-making role, relevant persona, and comes from a company well-aligned with B2B SAAS target industries and size. They must have a clear business need for data and AI services.'
        },{
        'label': 'Medium',
        'description': 'The lead is a good fit but may lack full decision-making authority or strong company characteristics. However, they should have influence or future potential.'
        },{
        'label': 'Low',
        'description': 'The lead has minimal alignment, lacks decision-making power, or is from a less relevant company. The persona or company factors do not strongly align with B2B SAAS ICP.'
        },{
        'label': 'Poor',
        'description': 'The lead has no alignment with B2B SAAS ICP. Their role and company characteristics are irrelevant. A form fill that appears to contain test accounts, junk, spam, scams, jokes, or nonsensical entries should be labeled as poor.'
        }
    ]

    --OPTIONS (A short explanation of the text classification task)
    , {'task_description': 'We are a B2B SAAS company. Return a classification the Ideal Customer Profile of this lead based on the text entered to a demand gen form on our website to register for a marketing event.'}
    
  ) as classification_output_raw
  , trim(classification_output_raw:label, '"') as classification_output
from AI_DEMO.MARKETING.MARKETING_LEADS d;

In [ ]:
CREATE OR REPLACE TABLE AI_DEMO.MARKETING.MARKETING_LEADS_SCORED AS

WITH ID_SPAM AS (
    select PERSON_ID
    , snowflake.cortex.CLASSIFY_TEXT(
    
        -- INPUT TEXT (concat several fields into a string)
        concat_ws(' '
        ,'First Name:', first_name
        ,'Last Name:', last_name
        ,'Job Title:', title
        ,'Company:', company )
            
        --CATEGORIES (Array that represents 2-100 categories)
        , [
            {
            'label': 'VALID',
            'description': 'A form fill that appears to be legitimate data from a valid persona.'
            },
            {
            'label': 'SPAM',
            'description': 'A form fill that appears to contain test accounts, junk, spam, scams, jokes, or nonsensical entries that cannot be worked by sales. This category should also include students and people without a valid company.'
            }
          ]
          
        --OPTIONS (A short explanation of the text classification task)
        , {'task_description': 'We are a B2B SAAS company. Return a classification for this text entered to a demand gen form on our website to register for a marketing event.'}
        
     ) as CLASSIFY_SPAM_RAW
      , trim(CLASSIFY_SPAM_RAW:label, '"') as SPAM_CD
     FROM AI_DEMO.MARKETING.MARKETING_LEADS
)

, ID_HARMFUL AS (
    select PERSON_ID
    , snowflake.cortex.CLASSIFY_TEXT(
    
        -- INPUT TEXT (concat several fields into a string)
        concat_ws(' '
        ,'First Name:', first_name
        ,'Last Name:', last_name
        ,'Job Title:', title
        ,'Company:', company )
    
        --CATEGORIES (Array that represents 2-100 categories)
        , [
            {
            'label': 'HARMFUL',
            'description': 'A form fill contains language not appropriate in a business context.'
            },{
            'label': 'NOT HARMFUL'
            }
          ]
    
        --OPTIONS (A short explanation of the text classification task)
        , {'task_description': 'We are a B2B SAAS company. Return a classification for this text entered to a demand gen form on our website to register for a marketing event.'}

     ) as CLASSIFY_HARMFUL_RAW
      , trim(CLASSIFY_HARMFUL_RAW:label, '"') as HARMFUL_CD
     FROM AI_DEMO.MARKETING.MARKETING_LEADS
)

, ID_TITLE AS (
    select PERSON_ID
    , company
    , snowflake.cortex.CLASSIFY_TEXT(

        -- INPUT TEXT (concat several fields into a string)
        concat_ws(' ','Job Title:', title )
        
        --CATEGORIES (Array that represents 2-100 categories)    
        , [
            {
            'label': 'CXO',
            'description': 'Any title that is a company C-level executive or founder.'
            },{
            'label': 'VP+',
            'description': 'Any title that is a company executive below C-level like a VP, President, or managing director.'
            },{
            'label': 'DIRECTOR',
            'description': 'Any title that is director level or head of a department.'
            },{
            'label': 'MANAGER',
            'description': 'Any title that relates to managers or team leads.'
            },{
            'label': 'IC',
            'description': 'Any title that relates to an individual contributor.'
            },{
            'label': 'JUNK',
            'description': 'Any title seems like junk, spam, scams, jokes, or nonsensical entries that cannot be worked by sales'
            }
          ]
    
        --OPTIONS (A short explanation of the text classification task)
        ,
          {'task_description': 'We are a B2B SAAS company. Use this data to classify job titles into seniority groupings. Consider all parts of the title and be careful of how the meaning changes based on parathesis or other punctuation.'}
          
      ) as CLASSIFY_TITLE_RAW
      , trim(CLASSIFY_TITLE_RAW:label, '"') as TITLE_CD
     FROM AI_DEMO.MARKETING.MARKETING_LEADS
)

, VALID_COMPANY AS (
    select PERSON_ID
    , TITLE_CD
    , snowflake.cortex.CLASSIFY_TEXT(

        -- INPUT TEXT (concat several fields into a string)
            concat_ws(' ','Company:', company )
        
        --CATEGORIES (Array that represents 2-100 categories)    
        , [
            {
            'label': 'VALID',
            'description': 'The company name appears to be valid.'
    
            },{
            'label': 'NOT VALID',
            'description': 'A company name seems like junk, spam, scams, jokes, or nonsensical entries that cannot be recognized by sales'
            }
          ]
    
        --OPTIONS (A short explanation of the text classification task)
        ,
          {'task_description': 'We are a B2B SAAS company. Use this data to classify provided company names as being likely valid or not.'}
          
      ) as CLASSIFY_COMPANY_RAW
      , trim(CLASSIFY_COMPANY_RAW:label, '"') as VALID_COMPANY_CD
      , CASE WHEN VALID_COMPANY_CD = 'VALID' THEN COMPANY ELSE 'UNKNOWN' END AS CLEANSED_COMPANY
    from ID_TITLE 
)

, PRIORITIZE_LEAD AS (
    select PERSON_ID
    , snowflake.cortex.CLASSIFY_TEXT(
    
        -- INPUT TEXT (concat several fields into a string)
        concat_ws(' '
            ,'Job Title:', TITLE_CD
            ,'Company:', CLEANSED_COMPANY
             )
            
        --CATEGORIES (Array that represents 2-100 categories)    
        ,[
            {
            'label': 'HIGH',
            'description': 'The lead title has a decision-making role or relevant persona. They comes from a company well-aligned with B2B SAAS target industries and size. They must have a clear business need for data and AI services.'
            },{
            'label': 'MEDIUM',
            'description': 'The lead title is a good fit but may lack full decision-making authority or strong company characteristics. However, they should have influence or future potential.'
            },{
            'label': 'LOW',
            'description': 'The lead title is an IC and is from a known company. Alternatively, the lead title is known, not an IC or JUNK, and is from an UNKNOWN company.'
            },{
            'label': 'POOR',
            'description': 'The lead title is JUNK and the company characteristics are UNKNOWN.'
            }
        ]
    
        --OPTIONS (A short explanation of the text classification task)
        , {'task_description': 'We are a B2B SAAS company. Return a classification the Ideal Customer Profile of this lead based on the text entered to a demand gen form on our website to register for a marketing event.'}
        
      ) as CLASSIFY_PRIORITIZE_RAW
      , trim(CLASSIFY_PRIORITIZE_RAW:label, '"') as PRIORITIZE_CD      
     FROM VALID_COMPANY

)

SELECT t1.*
, t2.SPAM_CD
, t3.HARMFUL_CD
, t4.TITLE_CD
, t5.CLEANSED_COMPANY
, t6.PRIORITIZE_CD
, CASE WHEN t2.SPAM_CD <> 'SPAM' 
        AND t3.HARMFUL_CD <> 'HARMFUL' 
        AND t4.TITLE_CD <> 'JUNK'
        AND t5.CLEANSED_COMPANY <> 'UNKNOWN'
        AND t6.PRIORITIZE_CD <> 'POOR'
        THEN 'QUALIFIED' ELSE 'NOT QUALIFIED' END AS QUALIFIED_LEAD_CD
, CASE WHEN QUALIFIED_LEAD_CD = 'QUALIFIED' AND t4.TITLE_CD = 'CXO' THEN 'URGENT' ELSE 'NOT URGENT' END AS URGENT_LEAD_CD
FROM AI_DEMO.MARKETING.MARKETING_LEADS t1
JOIN ID_SPAM t2 on t1.PERSON_ID = t2.PERSON_ID
JOIN ID_HARMFUL t3 on t1.PERSON_ID = t3.PERSON_ID
JOIN ID_TITLE t4 on t1.PERSON_ID = t4.PERSON_ID
JOIN VALID_COMPANY t5 on t1.PERSON_ID = t5.PERSON_ID
JOIN PRIORITIZE_LEAD t6 on t1.PERSON_ID = t6.PERSON_ID
;

In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import col, count
session = get_active_session()

# Load the MARKETING_LEADS_SCORED table into a Snowpark DataFrame
df = session.table("AI_DEMO.MARKETING.MARKETING_LEADS_SCORED")

# Group by the field XYZ and count the number of records
df = df.group_by(col("QUALIFIED_LEAD_CD"), col("URGENT_LEAD_CD")).agg(count("*").alias("LEAD_COUNT"))

st.bar_chart(df, x='QUALIFIED_LEAD_CD', y='LEAD_COUNT', color='URGENT_LEAD_CD')


In [ ]:
SELECT * EXCLUDE (PERSON_ID, URGENT_LEAD_CD)
FROM AI_DEMO.MARKETING.MARKETING_LEADS_SCORED
WHERE QUALIFIED_LEAD_CD = 'NOT QUALIFIED';